# Introduction to Edit Distance and Levenshtein Distance


Edit distance is a way of quantifying how dissimilar two strings are by counting the minimum number of operations required to transform one string into the other.

The three basic operations in the Levenshtein distance are:
1. **Insertion**: Add a character.
2. **Deletion**: Remove a character.
3. **Substitution**: Replace one character with another.
    

## Basic String Editing Examples

In [1]:

# Example strings
str1 = "cat"
str2 = "cut"

# Demonstrating edit operations
print("Original strings:")
print(f"String 1: {str1}")
print(f"String 2: {str2}\n")

# Substitution example
substituted = str1[:1] + "u" + str1[2:]
print(f"After substituting 'a' with 'u' in '{str1}': {substituted}")

# Insertion example
inserted = str1[:2] + "s" + str1[2:]
print(f"After inserting 's' into '{str1}': {inserted}")

# Deletion example
deleted = str2[:2]
print(f"After deleting 't' from '{str2}': {deleted}")
    

Original strings:
String 1: cat
String 2: cut

After substituting 'a' with 'u' in 'cat': cut
After inserting 's' into 'cat': cast
After deleting 't' from 'cut': cu


## Introducing Levenshtein Distance

In [2]:
def levenshtein_distance(str1, str2):
    """
    Calculate the minimum edit distance between two strings using the Levenshtein algorithm,
    with substitution cost set to 2.

    Parameters:
        str1 (str): The first string.
        str2 (str): The second string.

    Returns:
        int: The minimum edit distance between str1 and str2.
    """
    m, n = len(str1), len(str2)
    
    # Initialize a (m+1) x (n+1) matrix
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Fill the base cases
    for i in range(m + 1):
        dp[i][0] = i  # Cost of deleting all characters from str1
    for j in range(n + 1):
        dp[0][j] = j  # Cost of inserting all characters into str1

    # Compute the distances
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                cost = 0  # No cost if characters are the same
            else:
                cost = 2  # Substitution cost

            dp[i][j] = min(
                dp[i - 1][j] + 1,        # Deletion
                dp[i][j - 1] + 1,        # Insertion
                dp[i - 1][j - 1] + cost  # Substitution
            )

    # The final value in the matrix is the edit distance
    return dp[m][n]

# Example usage
str1 = "kitten"
str2 = "sitting"
distance = levenshtein_distance(str1, str2)
print(f"Minimum edit distance between '{str1}' and '{str2}': {distance}")


Minimum edit distance between 'kitten' and 'sitting': 5


## Step-by-Step Table Construction

In [3]:
# Example 1: Test case from user
str1 = "#EXECUTION"
str2 = "#INTENTION"
print(f"Edit distance between '{str1}' and '{str2}': {levenshtein_distance(str1, str2)}")

# Example 2: Simple strings
str1 = "kitten"
str2 = "sitting"
print(f"Edit distance between '{str1}' and '{str2}': {levenshtein_distance(str1, str2)}")

# Example 3: Identical strings
str1 = "python"
str2 = "python"
print(f"Edit distance between '{str1}' and '{str2}': {levenshtein_distance(str1, str2)}")

# Example 4: Completely different strings
str1 = "flaw"
str2 = "lawn"
print(f"Edit distance between '{str1}' and '{str2}': {levenshtein_distance(str1, str2)}")

# Example 5: One string is empty
str1 = "hello"
str2 = ""
print(f"Edit distance between '{str1}' and '{str2}': {levenshtein_distance(str1, str2)}")

# Example 6: Case-sensitive strings
str1 = "Hello"
str2 = "hello"
print(f"Edit distance between '{str1}' and '{str2}': {levenshtein_distance(str1, str2)}")


Edit distance between '#EXECUTION' and '#INTENTION': 8
Edit distance between 'kitten' and 'sitting': 5
Edit distance between 'python' and 'python': 0
Edit distance between 'flaw' and 'lawn': 2
Edit distance between 'hello' and '': 5
Edit distance between 'Hello' and 'hello': 2


In [4]:
# nltk min edit distance calculation
from nltk.metrics.distance import edit_distance

# Example 1: Test case from user
str1 = "#EXECUTION"
str2 = "#INTENTION"
print(f"Edit distance between '{str1}' and '{str2}': {edit_distance(str1, str2, substitution_cost=2)}")

Edit distance between '#EXECUTION' and '#INTENTION': 8


### Full example with backtrace

In [5]:
def levenshtein_distance_with_trace(str1, str2):
    """
    Calculate the minimum edit distance between two strings using the Levenshtein algorithm,
    with substitution cost set to 2. Also, generates a table of operations for tracing.

    Parameters:
        str1 (str): The first string.
        str2 (str): The second string.

    Returns:
        tuple: The minimum edit distance and the operation table.
    """
    m, n = len(str1), len(str2)
    
    # Initialize a (m+1) x (n+1) matrix
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    operation = [[""] * (n + 1) for _ in range(m + 1)]  # To trace operations

    # Fill the base cases
    for i in range(m + 1):
        dp[i][0] = i  # Cost of deleting all characters from str1
        operation[i][0] = "DEL" if i > 0 else ""  # Mark deletions
    for j in range(n + 1):
        dp[0][j] = j  # Cost of inserting all characters into str1
        operation[0][j] = "INS" if j > 0 else ""  # Mark insertions

    # Compute the distances
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if str1[i - 1] == str2[j - 1]:
                cost = 0  # No cost if characters are the same
                operation[i][j] = "COPY"
            else:
                cost = 2  # Substitution cost
                operation[i][j] = "SUB"

            # Select the minimum cost operation
            choices = [
                (dp[i - 1][j] + 1, "DEL"),         # Deletion
                (dp[i][j - 1] + 1, "INS"),         # Insertion
                (dp[i - 1][j - 1] + cost, "SUB")   # Substitution
            ]
            dp[i][j], operation[i][j] = min(choices)

    # Backtrace the operations
    i, j = m, n
    backtrace = []
    while i > 0 or j > 0:
        backtrace.append(operation[i][j])
        if operation[i][j] == "DEL":
            i -= 1
        elif operation[i][j] == "INS":
            j -= 1
        else:  # SUB or COPY
            i -= 1
            j -= 1
    backtrace.reverse()

    return dp[m][n], dp, operation, backtrace

# Example usage
str1 = "#INTENTION"
str2 = "#EXECUTION"
distance, dp_table, operation_table, backtrace = levenshtein_distance_with_trace(str1, str2)

print(f"Minimum edit distance between '{str1}' and '{str2}': {distance}")
print("\n#INTENTION: DP Table")
for row in dp_table:
    print(row)

print("\n#EXECUTION: Operation Table")
for row in operation_table:
    print(row)

print("\nBacktrace of operations:")
print(backtrace)


Minimum edit distance between '#INTENTION' and '#EXECUTION': 8

#INTENTION: DP Table
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[2, 1, 2, 3, 4, 5, 6, 7, 6, 7, 8]
[3, 2, 3, 4, 5, 6, 7, 8, 7, 8, 7]
[4, 3, 4, 5, 6, 7, 8, 7, 8, 9, 8]
[5, 4, 3, 4, 5, 6, 7, 8, 9, 10, 9]
[6, 5, 4, 5, 6, 7, 8, 9, 10, 11, 10]
[7, 6, 5, 6, 7, 8, 9, 8, 9, 10, 11]
[8, 7, 6, 7, 8, 9, 10, 9, 8, 9, 10]
[9, 8, 7, 8, 9, 10, 11, 10, 9, 8, 9]
[10, 9, 8, 9, 10, 11, 12, 11, 10, 9, 8]

#EXECUTION: Operation Table
['', 'INS', 'INS', 'INS', 'INS', 'INS', 'INS', 'INS', 'INS', 'INS', 'INS']
['DEL', 'SUB', 'INS', 'INS', 'INS', 'INS', 'INS', 'INS', 'INS', 'INS', 'INS']
['DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'SUB', 'INS', 'INS']
['DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'SUB']
['DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'DEL', 'SUB', 'DEL', 'DEL', 'DEL']
['DEL', 'DEL', 'SUB', 'INS', 'INS', 'INS', 'INS', 'DEL', 'DEL', 'DEL', 'DEL']
['DEL', 'DEL', 'DEL', 'DEL', 